In [1]:
import requests
import base64
import datetime
from urllib.parse import urlencode

In [2]:
client_id = '18bf732ea7ab485e91a45dfe7b75d5ec'
client_secret = '61d0f5271ab34e7aa519f1bebd81d3c9'

In [3]:
class SpotifyAPI(object):
    access_token = None
    access_token_expires = datetime.datetime.now()
    access_token_did_expire = True
    client_id = None
    client_secret = None
    token_url = "https://accounts.spotify.com/api/token"
    
    def __init__(self, client_id, client_secret, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.client_id = client_id
        self.client_secret = client_secret

    def get_client_credentials(self):
        """
        Returns a base64 encoded string
        """
        client_id = self.client_id
        client_secret = self.client_secret
        if client_secret == None or client_id == None:
            raise Exception("You must set client_id and client_secret")
        client_creds = f"{client_id}:{client_secret}"
        client_creds_b64 = base64.b64encode(client_creds.encode())
        return client_creds_b64.decode()
    
    def get_token_headers(self):
        client_creds_b64 = self.get_client_credentials()
        return {
            "Authorization": f"Basic {client_creds_b64}"
        }
    
    def get_token_data(self):
        return {
            "grant_type": "client_credentials"
        } 
    
    def perform_auth(self):
        token_url = self.token_url
        token_data = self.get_token_data()
        token_headers = self.get_token_headers()
        r = requests.post(token_url, data=token_data, headers=token_headers)
        if r.status_code not in range(200, 299):
            raise Exception("Could not authenticate client.")
            # return False
        data = r.json()
        now = datetime.datetime.now()
        access_token = data['access_token']
        expires_in = data['expires_in'] # seconds
        expires = now + datetime.timedelta(seconds=expires_in)
        self.access_token = access_token
        self.access_token_expires = expires
        self.access_token_did_expire = expires < now
        return True
    
    def get_access_token(self):
        token = self.access_token
        expires = self.access_token_expires
        now = datetime.datetime.now()
        if expires < now:
            self.perform_auth()
            return self.get_access_token()
        elif token == None:
            self.perform_auth()
            return self.get_access_token() 
        return token
    
    def get_resource_header(self):
        access_token = self.get_access_token()
        headers = {
            "Authorization": f"Bearer {access_token}"
        }
        return headers
        
        
    def get_resource(self, lookup_id, resource_type='albums', version='v1'):
        endpoint = f"https://api.spotify.com/{version}/{resource_type}/{lookup_id}"
        headers = self.get_resource_header()
        r = requests.get(endpoint, headers=headers)
        if r.status_code not in range(200, 299):
            return {}
        return r.json()
    
    def get_album(self, _id):
        return self.get_resource(_id, resource_type='albums')
    
    def get_artist(self, _id):
        return self.get_resource(_id, resource_type='artists')
    
    def search(self, query, search_type='artist' ): # type
        headers = self.get_resource_header()
        endpoint = "https://api.spotify.com/v1/search"
        data = urlencode({"q": query, "type": search_type.lower()})
        lookup_url = f"{endpoint}?{data}"
        r = requests.get(lookup_url, headers=headers)
        if r.status_code not in range(200, 299):  
            return {}
        return r.json()

In [4]:
spotify = SpotifyAPI(client_id, client_secret)

In [5]:
spotify.search("Time", search_type="track")

    'BE',
     'BG',
     'BH',
     'BO',
     'BR',
     'BY',
     'CA',
     'CH',
     'CL',
     'CO',
     'CR',
     'CY',
     'CZ',
     'DE',
     'DK',
     'DO',
     'DZ',
     'EC',
     'EE',
     'EG',
     'ES',
     'FI',
     'FR',
     'GB',
     'GR',
     'GT',
     'HK',
     'HN',
     'HR',
     'HU',
     'ID',
     'IE',
     'IL',
     'IS',
     'IT',
     'JO',
     'JP',
     'KW',
     'KZ',
     'LB',
     'LI',
     'LT',
     'LU',
     'LV',
     'MA',
     'MC',
     'MD',
     'ME',
     'MK',
     'MT',
     'MX',
     'MY',
     'NI',
     'NL',
     'NO',
     'NZ',
     'OM',
     'PA',
     'PE',
     'PH',
     'PL',
     'PS',
     'PT',
     'PY',
     'QA',
     'RO',
     'RS',
     'RU',
     'SA',
     'SE',
     'SG',
     'SI',
     'SK',
     'SV',
     'TH',
     'TN',
     'TR',
     'TW',
     'UA',
     'US',
     'UY',
     'VN',
     'XK',
     'ZA'],
    'disc_number': 1,
    'duration_ms': 146175,
    'explicit': False,
    

In [6]:
spotify.get_artist('2xiIXseIJcq3nG7C8fHeBj')

{'external_urls': {'spotify': 'https://open.spotify.com/artist/2xiIXseIJcq3nG7C8fHeBj'},
 'followers': {'href': None, 'total': 3792732},
 'genres': ['alternative metal',
  'canadian metal',
  'canadian rock',
  'nu metal',
  'pop punk',
  'post-grunge'],
 'href': 'https://api.spotify.com/v1/artists/2xiIXseIJcq3nG7C8fHeBj',
 'id': '2xiIXseIJcq3nG7C8fHeBj',
 'images': [{'height': 640,
   'url': 'https://i.scdn.co/image/2a9ec8d494f8d0a52fd67c3239efc2b9e79a4ced',
   'width': 640},
  {'height': 320,
   'url': 'https://i.scdn.co/image/4d8741f7fca53b7b1ff9d6d6b1176420df7e8200',
   'width': 320},
  {'height': 160,
   'url': 'https://i.scdn.co/image/aaeae81ffa892a04619203bfbab22ad0582c2e9d',
   'width': 160}],
 'name': 'Three Days Grace',
 'popularity': 76,
 'type': 'artist',
 'uri': 'spotify:artist:2xiIXseIJcq3nG7C8fHeBj'}

In [7]:
spotify.get_album("4ZDwEvXIKxWdwhAQK8ndBU")


     'MK',
     'MT',
     'MX',
     'MY',
     'NI',
     'NL',
     'NO',
     'NZ',
     'OM',
     'PA',
     'PE',
     'PH',
     'PL',
     'PS',
     'PT',
     'PY',
     'QA',
     'RO',
     'RS',
     'RU',
     'SA',
     'SE',
     'SG',
     'SI',
     'SK',
     'SV',
     'TH',
     'TN',
     'TR',
     'TW',
     'UA',
     'US',
     'UY',
     'VN',
     'XK',
     'ZA'],
    'disc_number': 1,
    'duration_ms': 202973,
    'explicit': False,
    'external_urls': {'spotify': 'https://open.spotify.com/track/5PZ2cqh9Yem2g6cTSOLllz'},
    'href': 'https://api.spotify.com/v1/tracks/5PZ2cqh9Yem2g6cTSOLllz',
    'id': '5PZ2cqh9Yem2g6cTSOLllz',
    'is_local': False,
    'name': 'Pain',
    'preview_url': 'https://p.scdn.co/mp3-preview/6a431555ca663924569dc0a5b5ba6635c7106f55?cid=18bf732ea7ab485e91a45dfe7b75d5ec',
    'track_number': 2,
    'type': 'track',
    'uri': 'spotify:track:5PZ2cqh9Yem2g6cTSOLllz'},
   {'artists': [{'external_urls': {'spotify': 'https://open.sp